In [ ]:
## general imports
import json
import numpy as np

## COCO imports
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from pycocotools.cocoanalyze import COCOanalyze

## plotting imports
%matplotlib inline
import matplotlib.pyplot as plt
import skimage.io as io

In [ ]:
## set paths
dataDir  = '.'
dataType = 'val2014'
annType  = 'keypoints'
teamName = 'fakekeypoints100'

annFile  = '%s/annotations/%s_%s.json'%(dataDir, annType, dataType)
resFile  = '%s/results/%s_%s_%s_results.json'%(dataDir, teamName, annType, dataType)

print("{:10}[{}]".format('annFile:',annFile))
print("{:10}[{}]".format('resFile:',annFile))

In [ ]:
## initialize COCO ground-truth api
coco_gt = COCO( annFile )
imgIds  = sorted(coco_gt.getImgIds())
imgIds  = imgIds[0:100]
imgId   = imgIds[np.random.randint(100)]

## initialize COCO detections api
coco_dt   = coco_gt.loadRes( resFile )

## initialize COCO analyze api
coco_analyze = COCOanalyze(coco_gt, coco_dt, annType)
coco_analyze.cocoEval.params.imgIds = imgIds

In [ ]:
# use evaluate() method for standard coco evaluation
# input arguments:
#  - verbose   : verbose outputs    (default: False)
#  - makeplots : plots eval results (default: False)
#  - savedir   : path to savedir    (default: None)
#  - team_name : team name string   (default: None)

coco_analyze.evaluate(verbose=True, makeplots=True)

In [ ]:
## NOTE: the values below are all default

# set OKS threshold of the extended error analysis
coco_analyze.params.oksThrs       = [.85]

# set OKS threshold required to match a detection to a ground truth
coco_analyze.params.oksLocThrs    = .1

# set KS threshold limits defining jitter errors
coco_analyze.params.jitterKsThrs = [.5,.85]

# set the localization errors to analyze and in what order
# note: different order will show different progressive improvement
# to study impact of single error type, study in isolation
coco_analyze.params.err_types = ['miss','swap','inversion','jitter']

# area ranges for evaluation
# 'all' range is union of medium and large
coco_analyze.params.areaRng       = [[32 ** 2, 1e5 ** 2]]#,[[32 ** 2, 96 ** 2]],[[96 ** 2, 1e5 ** 2]]
coco_analyze.params.areaRngLbl    = ['all']#,['medium'],['large']

In [ ]:
# use analyze() method for advanced error analysis 
# input arguments:
#  - check_kpts   : analyze keypoint localization errors for detections with a match (default: True)
#                 : errors types are ['jitter','inversion','swap','miss']
#  - check_scores : analyze optimal score (maximizing oks over all matches) for every detection (default: True)
#  - check_bkgd   : analyze background false positives and false negatives (default: True)

coco_analyze.analyze(check_kpts=True, check_scores=True, check_bkgd=True)

In [ ]:
## after analyze() has been called the following variables are available

# list of the corrected detections
corrected_dts = coco_analyze.corrected_dts

i = 17
# info on keypoint detection localization error
print 'good: %s'%corrected_dts[i]['good']
print 'miss: %s'%corrected_dts[i]['miss']
print 'swap: %s'%corrected_dts[i]['swap']
print 'inv.: %s'%corrected_dts[i]['inversion']
print 'jit.: %s\n'%corrected_dts[i]['jitter']

# corrected keypoint locations
print 'predicted keypoints:\n %s'%corrected_dts[i]['keypoints']
print 'corrected keypoints:\n %s\n'%corrected_dts[i]['opt_keypoints']

# optimal detection score
print 'original score: %s'%corrected_dts[i]['score']
print 'optimal score:  %s\n'%corrected_dts[i]['opt_score']

# dictionary with all detection and ground truth matches
dt_gt_matches = coco_analyze.matches
print "Dt-gt match:"
for k in dt_gt_matches['dts'][corrected_dts[i]['id']][0]:
    print k, dt_gt_matches['dts'][corrected_dts[i]['id']][0][k]

In [ ]:
# use summarize() method to get the results after progressive correction of errors
# input arguments:
#  - makeplots : plots eval results (default: False)
#  - savedir   : path to savedir    (default: None)
#  - team_name : team name string   (default: None)

coco_analyze.summarize(makeplots=True)

In [ ]:
## after summarize() has been called the following variables are available

# list of the missed ground-truth annotations
false_neg_gts = coco_analyze.false_neg_gts

In [ ]:
## list the performance summary 
for stat in coco_analyze.stats: print stat